In [4]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import numpy as np

#from openmmtools.integrators import BAOABIntegrator


In [2]:
import simtk.openmm as mm
#import simtk.openmm as mm
from openmm import app
from simtk.openmm import app
import simtk.unit as unit
import numpy as np
import mdtraj as md
from openmm import unit as u
import pdbfixer
from sys import stdout

In [5]:
pdb = PDBFile('/home/nikolasmif98/Backward-master_which_worked_shit/try_fix_FV/b2.pdb')  #loads pdb of water box
fixer = pdbfixer.PDBFixer('/home/nikolasmif98/Backward-master_which_worked_shit/try_fix_FV/b2.pdb')  #fixes missing molecules

fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.findNonstandardResidues()
fixer.addMissingAtoms()

In [12]:
fixer.topology
#type(fixer.positions)

<Topology; 1 chains, 2 residues, 30 atoms, 30 bonds>

In [3]:
class E_pot_reporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return (steps, False, False, False, True , None)

    def report(self, simulation, state):
        energy = state.getPotentialEnergy().value_in_unit(u.kilojoules/mole)
        for e in energy:
            self._out.write('%g %g %g\n' % (e[0]))


class E_kin_reporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return (steps, False, False, False, True , None)

    def report(self, simulation, state):
        energy = state.getKineticEnergy().value_in_unit(u.kilojoules/mole)
        for e in energy:
            self._out.write('%g %g %g\n' % (e[0]))

In [4]:
STRIDE_water = 1000
STRIDE_no_water = 1000 
STRIDE_checkpoint = 10000
simulations_steps = 100000 
real_time_STRIDE = 1000

geometry = 'truncatedOctahedron'
geompadding = 2*1.2*u.nanometer

platform_name = 'CPU'#'OpenCL' 

temperature = 300 * u.kelvin #paper
friction_coeff = 1.0/u.picoseconds
time_step = 1.0*u.femtoseconds 
pressure = 1 * u.bar #paper

In [5]:
pdb = PDBFile('waterbox.pdb')  #loads pdb of water box
fixer = pdbfixer.PDBFixer('waterbox.pdb')  #fixes missing molecules

fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.findNonstandardResidues()
fixer.addMissingAtoms()

#Set pH to neutral
fixer.addMissingHydrogens(pH=7.0)  #sets pH

## NVT ensemble equilibration

In [6]:

ff = app.ForceField('tip3p.xml')   #Force field

model = app.Modeller(fixer.topology, fixer.positions)   #sets position and model

model.addSolvent(ff,            #FF
             #padding=5*u.nanometer,
             boxSize=Vec3(3.0, 3.0, 3.0)*nanometers,        #Boxsize
             model='tip3p',                         #Solvent Model
             ionicStrength=0.1 * u.millimolar,      #ionic strenght (random)
             #numAdded=409,
            #  positiveIon='Na+',     
            #  negativeIon='Cl-',
            #  neutralize=True,
             #numAdded=109 
              )

app.PDBFile.writeFile(model.topology,model.positions, open('./data_openmm/WaterBox_eq1.pdb', 'w'))   #writes PDB with new Water box


system = ff.createSystem(model.topology, 
                                 nonbondedMethod=app.PME, 
                                 nonbondedCutoff=1.2*u.nanometers,#paper 
                                 #constraints=app.HBonds, #No H-constrains
                                 rigidWater=True, 
                                 ewaldErrorTolerance=0.0005)

#system.addForce(mm.MonteCarloBarostat(pressure, temperature)) #NPT-enseble

#Termostat

system.addForce(mm.AndersenThermostat(temperature,friction_coeff))


integrator = mm.VerletIntegrator(time_step)#,   #Change to NoseHooverIntegrator (Leap-frog),
                                        #time_step)

integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(model.topology, system, integrator, platform)

simulation.context.setPositions(model.positions)
#simulation.context.setVelocities(model.positions)

simulation.minimizeEnergy()

simulation.reporters.append(mm.app.dcdreporter.DCDReporter('data_openmm/output_waterB_NVT.dcd',
                                                           STRIDE_water))

simulation.reporters.append(mm.app.pdbreporter.PDBReporter('data_openmm/lastframe_NVT.pdb',
                                                           simulations_steps))

simulation.saveState('output.xml')

simulation.reporters.append(mm.app.statedatareporter.StateDataReporter('data_openmm/mmBox_NVT.log', 
                                                                       STRIDE_no_water, 
                                                                       step=True, 
                                                                       potentialEnergy=True, 
                                                                       temperature=True, 
                                                                       time=True))

simulation.reporters.append(mm.app.statedatareporter.StateDataReporter(stdout,
                                                                       real_time_STRIDE, 
                                                                       step=True, 
                                                                       potentialEnergy=True, 
                                                                       temperature=True, 
                                                                       speed=True, 
                                                                       elapsedTime=True))


simulation.reporters.append(mm.app.checkpointreporter.CheckpointReporter('data_openmm/checkpoint_NVT.chk',
                            STRIDE_checkpoint))




In [7]:
simulation.step(simulations_steps)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Elapsed Time (s)"
1000,-27403.195143441204,125.55630632707957,0,0.0001385211944580078
2000,-26104.89463303471,192.16366807992392,24.1,3.5890674591064453
3000,-25164.014777878765,231.0514944728987,27.9,6.198059320449829
4000,-24638.84929154301,255.41535207148092,30.3,8.56090784072876
5000,-24208.992561558727,257.15687499354334,30.9,11.186445236206055
6000,-24004.36093972111,277.010099660133,31,13.950337171554565
7000,-23747.22118256474,287.5114789373653,31,16.7423837184906
8000,-23656.34438870335,299.8114539791509,30.6,19.76612663269043
9000,-23608.46763680363,297.9419121662926,30.8,22.44277000427246
10000,-23582.0231349268,304.3696237345377,31.1,24.97142243385315
11000,-23502.461073616985,302.1516849133051,31.4,27.508689403533936
12000,-23496.71611855412,301.68396058034415,31.3,30.340027332305908
13000,-23429.56695435429,301.28977679337373,31,33.472206592559814
14000,-23435.086656311993,298.3088053351301,30.6,36.6

## NPT ensemble

In [8]:
STRIDE_water = 1000
STRIDE_no_water = 1000 
STRIDE_checkpoint = 10000
simulations_steps = 100000 
real_time_STRIDE = 1000

geometry = 'truncatedOctahedron'
geompadding = 2*1.2*u.nanometer

platform_name = 'CPU'#'OpenCL' 

temperature = 300 * u.kelvin #paper
friction_coeff = 1.0/u.picoseconds
time_step = 1.0*u.femtoseconds 
pressure = 1 * u.bar #paper

In [9]:
pdb = PDBFile('./data_openmm/lastframe_NVT.pdb')  #loads pdb of water box
fixer = pdbfixer.PDBFixer('./data_openmm/lastframe_NVT.pdb')  #fixes missing molecules

fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.findNonstandardResidues()
fixer.addMissingAtoms()

#Set pH to neutral
fixer.addMissingHydrogens(pH=7.0)  #sets pH

In [10]:

ff = app.ForceField('tip3p.xml')

model = app.Modeller(fixer.topology, fixer.positions)

# model.addSolvent(forcefield,
#              padding=1.2*u.nanometer,
#              model='tip3p', 
#              ionicStrength=0.1 * u.millimolar, 
#              positiveIon='Na+',
#              negativeIon='Cl-',
#              neutralize=True )

# model.addSolvent(ff,
#              padding=3*u.nanometer,
#              #boxSize=Vec3(3.0, 3.0, 3.0)*nanometers,
#              model='tip3p', 
#              ionicStrength=0.1 * u.millimolar, 
#              positiveIon='Na+',
#              negativeIon='Cl-',
#              neutralize=True,
#              #numAdded=1000 
#               )

#app.PDBFile.writeFile(model.topology,model.positions, open('data/WB.pdb', 'w'))


system = ff.createSystem(model.topology, 
                                 nonbondedMethod=app.PME, 
                                 nonbondedCutoff=1.2*u.nanometers,#paper 
                                 #constraints=app.HBonds, #No H-constrains
                                 rigidWater=True, 
                                 ewaldErrorTolerance=0.0005)

system.addForce(mm.MonteCarloBarostat(pressure, temperature)) #NPT-enseble

integrator = mm.VerletIntegrator(time_step)

integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(model.topology, system, integrator, platform)

simulation.context.setPositions(model.positions)

#simulation.context.setVelocities(model.velocities)

simulation.context.setVelocitiesToTemperature(temperature)#,randomSeed=-1000)
#simulation.context.setVelocities(model.positions)

#simulation.minimizeEnergy()

simulation.reporters.append(mm.app.dcdreporter.DCDReporter('data_openmm/output_waterB_NPT.dcd',
                                                           STRIDE_water))

simulation.reporters.append(mm.app.pdbreporter.PDBReporter('data_openmm/lastframe_NPT.pdb',
                                                           simulations_steps))

simulation.saveState('output.xml')

simulation.reporters.append(mm.app.statedatareporter.StateDataReporter('data_openmm/mmBox_NPT.log', 
                                                                       STRIDE_no_water, 
                                                                       step=True, 
                                                                       potentialEnergy=True, 
                                                                       temperature=True, 
                                                                       time=True))

simulation.reporters.append(mm.app.statedatareporter.StateDataReporter(stdout,
                                                                       real_time_STRIDE, 
                                                                       step=True, 
                                                                       potentialEnergy=True, 
                                                                       temperature=True, 
                                                                       speed=True, 
                                                                       elapsedTime=True))


simulation.reporters.append(mm.app.checkpointreporter.CheckpointReporter('data_openmm/checkpoint_NPT.chk',
                            STRIDE_checkpoint))




In [11]:
simulation.step(simulations_steps)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Elapsed Time (s)"
1000,-24061.28499213818,298.08233181708385,0,9.989738464355469e-05
2000,-24267.020756779104,304.6459635744461,23.5,3.683285713195801
3000,-24232.543037800526,295.97403658768263,24.8,6.977393388748169
4000,-24264.972973047912,293.7810002049178,24.7,10.484866619110107
5000,-24454.54413875009,303.753943442991,24.6,14.020774602890015
6000,-24543.371967007464,304.2386852155169,24.2,17.845824241638184
7000,-24582.99817115936,303.9747236287988,24.3,21.347897052764893
8000,-24695.170611964986,310.02843320541774,24.1,25.08141255378723
9000,-24663.48301110089,306.89019995566497,24,28.799700260162354
10000,-24721.019908511375,308.586357612953,23.8,32.67656087875366
11000,-24697.686139037363,307.280022674867,23.8,36.352760314941406
12000,-24617.739945141366,302.8289155841341,23.8,39.994518995285034
13000,-24621.96532437228,300.99216350795484,23.8,43.60423302650452
14000,-24671.948925133787,304.6198986114792,

## NVE ensemble

In [5]:
STRIDE_water = 10
STRIDE_no_water = 1000 
STRIDE_checkpoint = 10000
simulations_steps = 20000000 
real_time_STRIDE = 1000

geometry = 'truncatedOctahedron'
geompadding = 2*1.2*u.nanometer

platform_name = 'CPU'#'OpenCL' 

temperature = 300 * u.kelvin #paper
friction_coeff = 1.0/u.picoseconds
time_step = 0.1*u.femtoseconds 
pressure = 1 * u.bar #paper

In [6]:
pdb = PDBFile('./data_openmm/lastframe_NPT.pdb')  #loads pdb of water box
fixer = pdbfixer.PDBFixer('./data_openmm/lastframe_NPT.pdb')  #fixes missing molecules

fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.findNonstandardResidues()
fixer.addMissingAtoms()

#Set pH to neutral
fixer.addMissingHydrogens(pH=7.0) 

In [7]:


ff = app.ForceField('tip3p.xml')

model = app.Modeller(fixer.topology, fixer.positions)

# model.addSolvent(forcefield,
#              padding=1.2*u.nanometer,
#              model='tip3p', 
#              ionicStrength=0.1 * u.millimolar, 
#              positiveIon='Na+',
#              negativeIon='Cl-',
#              neutralize=True )

model.addSolvent(ff,
             #padding=3*u.nanometer,
             boxSize=Vec3(3.0, 3.0, 3.0)*nanometers,
             model='tip3p', 
             ionicStrength=0.1 * u.millimolar, 
             positiveIon='Na+',
             negativeIon='Cl-',
             neutralize=True,
             #numAdded=1000 
              )

app.PDBFile.writeFile(model.topology,model.positions, open('data_openmm/WB.pdb', 'w'))


system = ff.createSystem(model.topology, 
                                 nonbondedMethod=app.PME, 
                                 nonbondedCutoff=1.2*u.nanometers,#paper 
                                 #constraints=app.HBonds, #No H-constrains
                                 rigidWater=True, 
                                 ewaldErrorTolerance=0.0005)

#system.addForce(mm.MonteCarloBarostat(pressure, temperature)) #NPT-enseble

integrator = mm.VerletIntegrator(time_step)

integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(model.topology, system, integrator, platform)

simulation.context.setPositions(model.positions)

simulation.context.setVelocitiesToTemperature(temperature)#,randomSeed=-1000)


#simulation.context.setVelocities(model.velocities)

simulation.context.setVelocitiesToTemperature(temperature)#,randomSeed=-1000)
#simulation.context.setVelocities(model.positions)

#simulation.minimizeEnergy()

#simulation.reporters.append(E_kin_reporter('E_kin.txt', 1000))
#simulation.reporters.append(VelocityReporter('E_pot.txt', 1000))

simulation.reporters.append(mm.app.dcdreporter.DCDReporter('data_openmm/output_waterB_NVE.dcd',
                                                           STRIDE_water))

simulation.reporters.append(mm.app.pdbreporter.PDBReporter('data_openmm/lastframe_NVE.pdb',
                                                           simulations_steps))



simulation.saveState('output.xml')

simulation.reporters.append(mm.app.statedatareporter.StateDataReporter('data_openmm/mmBox_NVE_0_1_fs.log', 
                                                                       STRIDE_no_water, 
                                                                       step=True, 
                                                                       potentialEnergy=True, 
                                                                       temperature=True, 
                                                                       time=True))

simulation.reporters.append(mm.app.statedatareporter.StateDataReporter(stdout,
                                                                       real_time_STRIDE, 
                                                                       step=True, 
                                                                       potentialEnergy=True, 
                                                                       temperature=True, 
                                                                       speed=True, 
                                                                       elapsedTime=True))

simulation.reporters.append(mm.app.statedatareporter.StateDataReporter('energy.dcd',
                                                                       real_time_STRIDE, 
                                                                       step=True, 
                                                                       totalEnergy=True, 
                                                                       temperature=True))


simulation.reporters.append(mm.app.checkpointreporter.CheckpointReporter('data_openmm/checkpoint_NVE.chk',
                            STRIDE_checkpoint))



In [8]:


simulation.step(simulations_steps)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Elapsed Time (s)"
1000,-21395.519386809774,351.07121205828753,0,0.00010967254638671875
2000,-21403.968224090047,351.8079086604146,1.72,5.0190510749816895
3000,-21383.23756174416,350.18957783589246,1.92,9.022031545639038
4000,-21462.728366416402,355.34802475397044,1.99,13.040321111679077
5000,-21410.220875304647,352.1733430472579,2.06,16.7614643573761
6000,-21570.72438864083,362.34223088288485,2.12,20.42216730117798
7000,-21513.95078305573,358.78896255585033,2.14,24.247774362564087
8000,-21605.376979392477,364.4747875270107,2.14,28.304481983184814
9000,-21550.63703970284,361.12979617359343,2.17,31.89293336868286
10000,-21506.707047980733,358.26250355441067,2.18,35.58885169029236
11000,-21454.25542215676,354.8832249506845,2.21,39.157687187194824
12000,-21637.351347011037,366.486342996539,2.17,43.73904609680176
13000,-21698.98174433083,370.37053226913014,2.18,47.48574900627136
14000,-21730.281260055013,372.3843858263

122000,-21674.5312085566,368.89622005482425,2.24,467.058012008667
123000,-21570.14465956063,362.2867183291771,2.24,470.7437083721161
124000,-21455.441122573324,354.9497446510202,2.24,475.2983148097992
125000,-21536.78159098,360.1895040139437,2.24,478.86531496047974
126000,-21661.71896890969,368.1011844052889,2.24,482.4522442817688
127000,-21634.060028594442,366.39548723534944,2.24,486.10769748687744
128000,-21530.36878065438,359.76649308387795,2.24,490.53241777420044
129000,-21432.432934325643,353.59736928751556,2.24,494.21198773384094
130000,-21295.30986646981,344.78051068604424,2.24,497.81952238082886
131000,-21627.519980948873,365.8954435722404,2.24,501.5764274597168
132000,-21575.35568575234,362.6579375648855,2.24,505.98490500450134
133000,-21762.426929038473,374.54810908002116,2.24,509.9177134037018
134000,-21461.108730357595,355.3807967069408,2.24,513.4414789676666
135000,-21713.301752608724,371.35251208284706,2.24,517.06618475914
136000,-21712.698121589132,371.33259522655277,2.2

243000,-21562.478053611227,361.7562358596942,2.22,940.2471072673798
244000,-21611.529444259115,364.76313496862514,2.22,943.8999619483948
245000,-21617.256605666586,365.23745173066794,2.22,947.5362646579742
246000,-21716.011322539754,371.50866528057776,2.23,951.2528855800629
247000,-21533.13664202065,359.8210841118921,2.22,955.7264115810394
248000,-21538.448057692953,360.18198543772417,2.22,959.3751266002655
249000,-21752.275150817342,373.67750976965294,2.22,963.0715465545654
250000,-21718.129977744527,371.57774900272943,2.23,966.7792754173279
251000,-21943.853750747152,385.84022480698445,2.22,971.0354676246643
252000,-21778.310182136007,375.4706327065614,2.22,974.7841153144836
253000,-21864.320567649313,380.7966000540399,2.23,978.4715445041656
254000,-21702.00007776589,370.55277856331367,2.23,982.1356718540192
255000,-21715.024428885885,371.38287903633864,2.22,986.6604566574097
256000,-21768.741927188345,374.87753193396793,2.22,990.2585959434509
257000,-21888.491462272115,382.401611574

363000,-21530.255751651235,359.6890087163922,2.23,1403.940646648407
364000,-21631.859681647726,366.28420103965914,2.23,1407.6346695423126
365000,-21558.98388914437,361.5667097764426,2.23,1411.2675268650055
366000,-21799.60973600716,376.84971655745414,2.23,1414.9770770072937
367000,-21691.948768657156,369.8552105174471,2.23,1419.5889654159546
368000,-21663.446938079305,368.17779962873203,2.23,1423.2040390968323
369000,-21797.676677268453,376.6157746837881,2.23,1426.9170706272125
370000,-21659.414538901754,367.90835939408163,2.23,1430.5685360431671
371000,-21569.584230941244,362.19413441202386,2.23,1435.463354587555
372000,-21658.877836745687,367.94395825984014,2.23,1439.1198811531067
373000,-21764.336813491293,374.49317601975775,2.23,1442.7917449474335
374000,-21633.965965789266,366.15665407940963,2.23,1446.3899269104004
375000,-21725.388310950704,372.079436439025,2.23,1450.8103024959564
376000,-21755.910246413656,373.91630845704424,2.23,1454.5201778411865
377000,-21628.755870383688,365

483000,-21642.8032498774,366.9373087564672,2.23,1871.1984963417053
484000,-21592.229697745748,363.705465151034,2.23,1875.0385994911194
485000,-21390.75000242562,350.89044020012324,2.23,1878.7129051685333
486000,-21576.32931379647,362.7675321863096,2.23,1883.133686542511
487000,-21476.29276613564,356.2540540848791,2.23,1886.7935338020325
488000,-21672.11353544564,368.7560069497669,2.23,1890.4891979694366
489000,-21456.727062743612,354.8980220334323,2.23,1894.2216515541077
490000,-21796.358849090047,376.5391817048039,2.22,1898.924709558487
491000,-21572.15430216164,362.36512272736576,2.23,1902.5453915596008
492000,-21648.69135145516,367.1950676998656,2.23,1906.3315589427948
493000,-21689.712313216634,369.8728088973587,2.23,1909.997606754303
494000,-21787.191904586263,375.86661209163003,2.23,1914.2838189601898
495000,-21594.49042276711,363.7145224180398,2.23,1918.071571111679
496000,-21738.55399660439,372.8152282325486,2.23,1921.7599852085114
497000,-21598.258057158895,363.91699030847803,

...

19523000,-21748.523336928793,373.51049168316337,1.86,90629.48527908325
19524000,-21742.03266004891,373.17556139799336,1.86,90633.63049387932
19525000,-21773.995626967855,375.1849926920168,1.86,90637.81020188332
19526000,-21933.609163802572,385.28333877976036,1.86,90642.01153922081
19527000,-21637.23877195687,366.5089968069364,1.86,90646.33555316925
19528000,-21625.096043151327,365.5508833810763,1.86,90651.16810798645
19529000,-21550.198853057333,360.95637481391225,1.86,90655.775660038
19530000,-21749.23122267098,373.6307164306469,1.86,90659.79977488518
19531000,-21919.700912993856,384.41818336241437,1.86,90664.82831096649
19532000,-21768.951849502035,374.83093973050626,1.86,90669.35104560852
19533000,-21667.49272970528,368.491961393865,1.86,90673.18942713737
19534000,-21692.596425574728,370.03462716504646,1.86,90678.40444278717
19535000,-21704.87055067391,370.86888886804445,1.86,90683.40567994118
19536000,-21507.937428039022,358.31913253784603,1.86,90688.27059006691
19537000,-21757.731

19641000,-21755.66149095864,373.9890909032602,1.86,91155.1428744793
19642000,-21561.74032739968,361.757557554432,1.86,91158.85527849197
19643000,-21715.52311090798,371.5150148254335,1.86,91164.39149570465
19644000,-21613.88265852303,364.95364533607847,1.86,91168.60163021088
19645000,-21676.64713434548,369.05469064143824,1.86,91172.21416115761
19646000,-21794.377628844686,376.5366679769777,1.86,91176.95513272285
19647000,-21668.679043334432,368.5021477674034,1.86,91181.90427708626
19648000,-21507.307972472616,358.33903189370153,1.86,91186.62570500374
19649000,-21483.432621520467,356.8282682579776,1.86,91190.68318867683
19650000,-21490.624204677053,357.2517897104717,1.86,91195.83719444275
19651000,-21637.28535799355,366.47897065001695,1.86,91199.56250214577
19652000,-21682.696533721395,369.32222589289825,1.86,91203.65303826332
19653000,-21863.374970954366,380.78783496678045,1.86,91207.47159433365
19654000,-21821.686610263296,378.1911608486959,1.86,91212.36944270134
19655000,-21692.533068

19759000,-21707.44993070931,370.87975961460825,1.86,91676.41831231117
19760000,-21505.957024138876,358.16434092852495,1.86,91680.64534544945
19761000,-21447.595741790243,354.4782447514535,1.86,91684.63482761383
19762000,-21705.653687995382,370.8995724556766,1.86,91688.85297656059
19763000,-21704.97720388741,370.83866221605854,1.86,91693.60640668869
19764000,-21666.70369963021,368.3663422830335,1.86,91697.81553816795
19765000,-21744.1370263514,373.1911515449291,1.86,91702.26787543297
19766000,-21655.890487235494,367.6714274783529,1.86,91706.87958550453
19767000,-21875.077543776937,381.5966618093227,1.86,91710.85857582092
19768000,-21724.40929750771,371.99449568917197,1.86,91715.09280872345
19769000,-21499.584270995565,357.7294706094124,1.86,91718.79686307907
19770000,-21734.381384414144,372.5487456899489,1.86,91723.9734070301
19771000,-21424.500786345907,353.00858557675025,1.86,91728.44931292534
19772000,-21696.507738154836,370.22379812511423,1.86,91732.55568146706
19773000,-21489.69301

19877000,-21538.575643104024,360.2214058616241,1.86,92192.36598849297
19878000,-21715.77241091103,371.4430711544504,1.86,92196.82048225403
19879000,-21698.79899648995,370.39051163105756,1.86,92201.42078948021
19880000,-21962.78178552956,387.09407696327116,1.86,92206.16647863388
19881000,-21780.57910208077,375.61122893665816,1.86,92210.3827188015
19882000,-21787.39462904305,376.0347078068705,1.86,92214.70360660553
19883000,-21747.746625464864,373.5120462636291,1.86,92219.19164347649
19884000,-21827.782724898763,378.6099795885298,1.86,92222.76906561852
19885000,-21838.158154052206,379.34964243455335,1.86,92226.73670601845
19886000,-21719.390600722738,371.70243104413424,1.86,92230.6350004673
19887000,-21616.3285570559,365.20485054989587,1.86,92235.6334412098
19888000,-21822.49759249062,378.23112228109466,1.86,92239.7639951706
19889000,-21786.771850150533,375.9271105406297,1.86,92243.79295825958
19890000,-21408.790532153555,351.9638012798294,1.86,92247.79120731354
19891000,-21589.917796653

19995000,-21853.091843169637,380.18392490723653,1.86,92717.57751202583
19996000,-21674.333049815603,368.81072187908023,1.86,92721.49763131142
19997000,-21786.989183467336,375.9978837496027,1.86,92725.81776547432
19998000,-21865.194242042013,380.94988482268406,1.86,92730.53222942352
19999000,-21719.905264419027,371.86197694935265,1.86,92734.7036831379
20000000,-21768.9606175837,374.7920821169076,1.86,92739.2622768879


In [16]:
class E_pot_reporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return (steps, False, False, False, True , None)

    def report(self, simulation, state):
        energy = state.getPotentialEnergy().value_in_unit(u.kilojoules/u.mole)
        for e in energy:
            self._out.write('%g %g %g\n' % (e[0]))


class E_kin_reporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return (steps, False, False, False, True , None)

    def report(self, simulation, state):
        energy = state.getKineticEnergy().value_in_unit(u.kilojoules/u.mole)
        for ek in energy:
            self._out.write('%g %g %g\n' % (ek[0]))

In [55]:
class VelocityReporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return (steps, False, True, False, False, None)

    def report(self, simulation, state):
        velocities = state.getKineticEnergy().value_in_unit(u.kilojoules/u.mole)
        for v in velocities:
            self._out.write('%g %g %g\n' % (v[0], v[1], v[2]))

In [47]:
class VelocityReporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return (steps, False, True, False, False, None)

    def report(self, simulation, state):
        velocities = state.getVelocities().value_in_unit(u.nanometer/u.picoseconds)
        for v in velocities:
            self._out.write('%g %g %g\n' % (v[0], v[1], v[2]))

In [36]:
class ForceReporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return (steps, False, False, True, False, None)

    def report(self, simulation, state):
        forces = state.getForces().value_in_unit(u.kilojoules/u.mole/u.nanometer)
        for f in forces:
            self._out.write('%g %g %g\n' % (f[0], f[1], f[2]))

NameError: name 'model' is not defined

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
1000,-20158.070087845677,298.68948443383107,14.999779685528004
2000,-20009.06736224448,302.13408423314195,14.999779685528004
3000,-19961.43845790183,294.6729465519934,14.999779685528004
4000,-19997.243502076024,287.61752413931066,14.999779685528004
5000,-19936.462913926,305.80269780506825,14.999779685528004
6000,-20094.627953942174,309.3157061891479,14.999779685528004
7000,-19962.102886612767,294.4024501296884,14.999779685528004
8000,-20162.778170044774,307.83017768248936,14.999779685528004
9000,-20247.83424609458,294.66174230692945,14.999779685528004
10000,-20248.24954360282,301.64179204310324,14.999779685528004


2

SyntaxError: invalid syntax (1210205007.py, line 1)

Signature:
model.addSolvent(
    forcefield,
    model='tip3p',
    boxSize=None,
    boxVectors=None,
    padding=None,
    numAdded=None,
    positiveIon='Na+',
    negativeIon='Cl-',
    ionicStrength=Quantity(value=0, unit=molar),
    neutralize=True,
)
Docstring:
Add solvent (both water and ions) to the model to fill a rectangular box.

The algorithm works as follows:

1. Water molecules are added to fill the box.
2. Water molecules are removed if their distance to any solute atom is less than the sum of their van der Waals radii.
3. If the solute is charged and neutralize=True, enough positive or negative ions are added to neutralize it.  Each ion is added by
   randomly selecting a water molecule and replacing it with the ion.
4. Ion pairs are added to give the requested total ionic strength.  Note that only monovalent ions are currently supported.

The box size can be specified in any of several ways:

1. You can explicitly give the vectors defining the periodic box to use.
2. 